In [2]:
import qiskit as q
%matplotlib inline

circuit = q.QuantumCircuit(2, 2)
circuit.x(0)
circuit.cx(0,1)
circuit.measure([0,1], [0,1])
circuit.draw()

┌───┐     ┌─┐   
q_0: ┤ X ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1

In [4]:
from qiskit_ibm_runtime import QiskitRuntimeService

tk = "9fa99827d4bbc7985a4104acee64b21015204b44974bbfccba3fc13f50151ab7dd4656f430e2fad89ccc5b555068eabcc00c8fba876ad86b5aed5436c8cda69a"

In [6]:
service = QiskitRuntimeService(channel="ibm_quantum", token=tk)

In [8]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(channel="ibm_quantum", token=tk, set_as_default=True, overwrite=True)
 
service = QiskitRuntimeService()

In [137]:
service.backends()

[<IBMBackend('ibm_brisbane')>,
 <IBMBackend('ibm_kyiv')>,
 <IBMBackend('ibm_sherbrooke')>]

In [123]:
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [139]:
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
import numpy as np
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp
import random

shots_list = [1, 50, 500, 1024, 4000, 10000]
x = 0
while x != 200:
    r1 = random.randint(1, 65)
    r2 = random.choice(shots_list)
    
    circuit = QuantumCircuit(r1)
    circuit.measure_all()
     
    service = QiskitRuntimeService()
    backend = service.least_busy(operational=True, simulator=False)
    
    sampler = Sampler(backend)
    job = backend.run(circuit, shots=r2)
    result = job.result()
    x += 1

IBMBackendApiError: 'Error submitting job: \'403 Client Error: Forbidden for url: https://api.quantum.ibm.com/runtime/jobs. {"errors":[{"message":"Job create exceeds open plan job usage limits","code":4317,"solution":"Please wait until the beginning of next month to submit more jobs when your quota will reset.","more_info":"https://docs.quantum-computing.ibm.com/errors"}]}\''

In [98]:
import random

my_list = [1, 50, 500, 1024, 4000]
x = 0
while x != 5:
    print("Hi")
    r = random.randint(1, 5)
    r2 = random.choice(my_list)
    print(r2)
    x+=1

Hi
500
Hi
1024
Hi
4000
Hi
1024
Hi
1024


In [714]:
from qiskit import QuantumCircuit

# Create a quantum circuit for teleportation
qc = QuantumCircuit(3)
qc.h(1)  # Create entanglement
qc.cx(1, 2)  # Entangle qubits 1 and 2

qc.measure_all()
qc.draw()

░ ┌─┐      
   q_0: ───────────░─┤M├──────
        ┌───┐      ░ └╥┘┌─┐   
   q_1: ┤ H ├──■───░──╫─┤M├───
        └───┘┌─┴─┐ ░  ║ └╥┘┌─┐
   q_2: ─────┤ X ├─░──╫──╫─┤M├
             └───┘ ░  ║  ║ └╥┘
meas: 3/══════════════╩══╩══╩═
                      0  1  2

In [29]:
import pandas as pd

data = pd.read_csv("Zaosheng.csv")
data = data.drop("Noise", axis=1)
data

,Shots,Qubits,Frequency Up or Down,Noise > F.U.o.D.
0,1024,47,601,False
1,50,26,42,False
2,1,1,1,False
3,50,7,47,False
4,10000,48,1588,True
...,...,...,...,...
187,4000,5,3779,False
188,4000,3,3977,False
189,10000,2,9961,False
190,1,2,1,False


In [175]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = data.drop(columns=["Noise > F.U.o.D."])
y = data["Noise > F.U.o.D."]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 30)


model = DecisionTreeClassifier(max_depth = 19, min_samples_leaf = 4, min_samples_split = 11)
model.fit(X_train.values, y_train)

### [[shots, qubits, up or down]] ###

predictions = model.predict([[1024, 1, 16], [10000, 100, 500], [100, 100, 16], [1024, 1, 40], [1, 1, 1]])
predictions1 = model.predict(X_test)
score = accuracy_score(y_test, predictions1)
print(predictions, f"\nAccuracy: {score}")

[False  True  True False False] 
Accuracy: 0.8793103448275862


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


In [33]:
from sklearn.datasets import fetch_california_housing
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# Define the function to optimize using cross-validation
def dtree_cv(max_depth, min_samples_split, min_samples_leaf):
    # Define the model with the parameters to be optimized
    estimator = DecisionTreeRegressor(
        max_depth=int(max_depth),
        min_samples_split=int(min_samples_split),
        min_samples_leaf=int(min_samples_leaf),
        random_state=2
    )
    cval = cross_val_score(estimator, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
    return cval.mean() # The optimizer tries to maximize the function, so we negate the score to minimize it

# Define the parameter bounds
param_bounds = {
    'max_depth': (1, 20),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 20)
}

optimizer = BayesianOptimization(
    f=dtree_cv,
    pbounds=param_bounds,
    random_state=1,
)

optimizer.maximize(n_iter=25, init_points=5) # Bayesian optimization
best_params_bayes = optimizer.max['params']
best_params_bayes['max_depth'] = int(best_params_bayes['max_depth'])
best_params_bayes['min_samples_split'] = int(best_params_bayes['min_samples_split'])
best_params_bayes['min_samples_leaf'] = int(best_params_bayes['min_samples_leaf'])
best_score_bayes = optimizer.max['target']

print(f"Best Parameters (Bayesian Optimization): {best_params_bayes}")
print(f"Best Score (Bayesian Optimization): {best_score_bayes}")

|   iter    |  target   | max_depth | min_sa... | min_sa... |
-------------------------------------------------------------
| 1         | -0.1724   | 8.923     | 14.69     | 2.002     |
| 2         | -0.1505   | 6.744     | 3.788     | 3.662     |
| 3         | -0.1515   | 4.539     | 7.566     | 9.142     |
| 4         | -0.1823   | 11.24     | 8.965     | 14.33     |
| 5         | -0.1681   | 4.885     | 17.68     | 2.493     |
| 6         | -0.1681   | 2.498     | 4.105     | 6.151     |
| 7         | -0.1693   | 11.11     | 15.96     | 5.148     |
| 8         | -0.1436   | 6.839     | 5.769     | 4.597     |
| 9         | -0.1572   | 10.09     | 6.188     | 5.083     |
| 10        | -0.1639   | 5.782     | 7.913     | 2.774     |
| 11        | -0.1436   | 6.913     | 5.392     | 7.159     |
| 12        | -0.1467   | 9.049     | 2.153     | 7.974     |
| 13        | -0.1427   | 6.232     | 2.624     | 10.84     |
| 14        | -0.1518   | 8.297     | 1.101     | 13.55     |
| 15    